In [1]:
import pandas as pd

import numpy as np

# import matplotlib.pyplot as plt
# import seaborn as sns
# sns.set(context = 'notebook', #mostly controls relative sizes of things on plot 
#                         #The base context is “notebook”, and the other contexts are “paper”, “talk”, and “poster”
#         style = 'darkgrid', #dict, None, or one of {darkgrid, whitegrid, dark, white, ticks}
#         palette = 'deep', # Should be something that color_palette() can process.
#         font_scale = 1, 
#         color_codes = False, 
#         rc = None)

# from IPython.core.interactiveshell import InteractiveShell 
# InteractiveShell.ast_node_interactivity = 'last_expr' 
# setting = "all" allows multiple outputs to be displayed for a given input cell. don't use w plotting!

from IPython.display import display

# %matplotlib notebook
#%matplotlib inline

pd.__version__ , np.__version__ #,  matplotlib.__version__, sns.__version__

('0.23.0', '1.14.3')

In [2]:
from sklearn.model_selection import cross_val_score, GridSearchCV #KFold, StratifiedKFold,

from sklearn.metrics import roc_auc_score


from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.neural_network import MLPClassifier

In [3]:
pwd

'/home/thedohn/Notebooks'

In [4]:
% cd '/home/thedohn/Notebooks'

/home/thedohn/Notebooks


In [5]:
from Home_Credit_package.master_pipeline import master_pipeline

# load this

## load df

In [6]:
from functools import reduce

df_to_merge_0 = pd.read_csv('application_train_df_cleaned_final.csv', index_col = 'SK_ID_CURR')
df_to_merge_1 = pd.read_csv('bureau_df_aggregated_wrangled_final.csv', index_col = 'SK_ID_CURR')
df_to_merge_2 = pd.read_csv('installments_payments_wrangled_df_final.csv', index_col = 'SK_ID_CURR')

dfs = (df_to_merge_0,
       df_to_merge_1,
       df_to_merge_2,
       )

# these all have the same index (SK_ID_CURR) so no risk for funny business
total_df = reduce( lambda left, right: pd.merge( left, right, left_index = True, right_index = True), dfs)
# from https://stackoverflow.com/questions/23668427/pandas-joining-multiple-dataframes-on-columns

del dfs
del df_to_merge_0
del df_to_merge_1
del df_to_merge_2

In [7]:
total_df.info(verbose = True, null_counts = True);

<class 'pandas.core.frame.DataFrame'>
Int64Index: 307511 entries, 100002 to 456255
Data columns (total 86 columns):
TARGET                               307511 non-null int64
NAME_CONTRACT_TYPE                   307511 non-null object
CODE_GENDER                          307511 non-null object
FLAG_OWN_CAR                         307511 non-null object
FLAG_OWN_REALTY                      307511 non-null object
CNT_CHILDREN                         307511 non-null int64
NAME_EDUCATION_TYPE                  307511 non-null object
NAME_FAMILY_STATUS                   307511 non-null object
DAYS_BIRTH                           307511 non-null int64
FLAG_EMP_PHONE                       307511 non-null int64
FLAG_WORK_PHONE                      307511 non-null int64
FLAG_PHONE                           307511 non-null int64
FLAG_EMAIL                           307511 non-null int64
CNT_FAM_MEMBERS                      307511 non-null int64
REGION_RATING_CLIENT                 307511 non-null

In [8]:
# CHECK: this should be empty if everything is non null
total_df.isnull().any()[total_df.isnull().any()==True]

Series([], dtype: bool)

In [9]:
# # split into train, test, validate
# credit_train_and_validate_df, credit_test_df = train_test_split(total_df, random_state = 0, train_size = 8/10, test_size = 2/10)

# len(credit_train_and_validate_df)/len(total_df), len(credit_test_df)/len(total_df)

In [10]:
# credit_train_and_validate_df['TARGET'].value_counts()

In [11]:
# credit_train_and_validate_df.shape

In [12]:
# # split into train, test, validate
# credit_train_df, credit_validate_df = train_test_split(credit_train_and_validate_df, random_state = 0, train_size = 6/8, test_size = 2/8)

# len(credit_train_df)/len(total_df), len(credit_validate_df)/len(total_df)

In [13]:
# # for now
# del total_df
# del credit_test_df

# new models to run

In [14]:
1+1

2

### DONE. random forest random grid search, EXT sources, with poly interactions

In [15]:
total_df_piped, final_feature_list, total_pipeline, trans_list = \
master_pipeline(df_in = total_df[['EXT_SOURCE_1','EXT_SOURCE_2',
                                         'EXT_SOURCE_3']],
                int_cutoff=20, 
                poly_deg=4, 
                feats_with_interaction=['EXT_SOURCE_1','EXT_SOURCE_2',
                                         'EXT_SOURCE_3']
                )


Total original feat len is 3. Sum of feats is 3.
Final array is length 35. final feature list is length 35.


In [16]:
param_dist_dict =  { 'max_depth' :         [6,7,8],
                     'min_samples_leaf' :  [8,9,10,11,12],
                     'min_samples_split' : [2], 
                     'n_estimators':       [50]
                   }


forest_reg = RandomForestClassifier(random_state=0,
                                    class_weight = None)

my_grid_search = GridSearchCV(forest_reg, 
                                param_dist_dict, 
                                cv = 3,
                                return_train_score = True , 
                                scoring = 'roc_auc',
                                n_jobs = -1, 
                                refit = True,
                                verbose = 51, 
                                       )

my_grid_search.fit(total_df_piped, total_df['TARGET'])

Fitting 3 folds for each of 15 candidates, totalling 45 fits
[CV] max_depth=6, min_samples_leaf=8, min_samples_split=2, n_estimators=50 
Memmaping (shape=(307511, 35), dtype=float64) to new file /dev/shm/joblib_memmaping_pool_6234_140079007263768/6234-140079004598056-b675758a20e22f3b4e24b10b420720d1.pkl
Memmaping (shape=(307511,), dtype=int64) to new file /dev/shm/joblib_memmaping_pool_6234_140079007263768/6234-140079004598056-36f41ebc26c9047822d0779f2962f9e3.pkl
Memmaping (shape=(307511,), dtype=int64) to new file /dev/shm/joblib_memmaping_pool_6234_140079007263768/6234-140079004598056-7fee98903a11851f0c36b343c7ef5338.pkl
Memmaping (shape=(205007,), dtype=int64) to new file /dev/shm/joblib_memmaping_pool_6234_140079007263768/6234-140079004598056-981b55e712abb5a5a2369212f2856f0a.pkl
[CV] max_depth=6, min_samples_leaf=8, min_samples_split=2, n_estimators=50 
Memmaping (shape=(205007,), dtype=int64) to new file /dev/shm/joblib_memmaping_pool_6234_140079007263768/6234-140079004598056-52b4

[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:  4.2min
[CV]  max_depth=7, min_samples_leaf=11, min_samples_split=2, n_estimators=50, score=0.7288023822657059, total= 1.1min
[CV] max_depth=8, min_samples_leaf=9, min_samples_split=2, n_estimators=50 
[Parallel(n_jobs=-1)]: Done  27 tasks      | elapsed:  4.3min
[CV]  max_depth=7, min_samples_leaf=11, min_samples_split=2, n_estimators=50, score=0.7231330602675574, total= 1.1min
[CV] max_depth=8, min_samples_leaf=9, min_samples_split=2, n_estimators=50 
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:  4.3min
[CV]  max_depth=7, min_samples_leaf=12, min_samples_split=2, n_estimators=50, score=0.7245966067302967, total= 1.2min
[CV] max_depth=8, min_samples_leaf=10, min_samples_split=2, n_estimators=50 
[CV]  max_depth=7, min_samples_leaf=12, min_samples_split=2, n_estimators=50, score=0.7228385870649567, total= 1.1min
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done  30 tasks      | elapse

GridSearchCV(cv=3, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_depth': [6, 7, 8], 'min_samples_leaf': [8, 9, 10, 11, 12], 'min_samples_split': [2], 'n_estimators': [50]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=51)

In [17]:
pd.set_option('display.max_colwidth', -1)
pd.DataFrame(my_grid_search.cv_results_)[['params','mean_test_score']]

,params,mean_test_score
0,"{'max_depth': 6, 'min_samples_leaf': 8, 'min_samples_split': 2, 'n_estimators': 50}",0.725322
1,"{'max_depth': 6, 'min_samples_leaf': 9, 'min_samples_split': 2, 'n_estimators': 50}",0.725344
2,"{'max_depth': 6, 'min_samples_leaf': 10, 'min_samples_split': 2, 'n_estimators': 50}",0.725379
3,"{'max_depth': 6, 'min_samples_leaf': 11, 'min_samples_split': 2, 'n_estimators': 50}",0.725324
4,"{'max_depth': 6, 'min_samples_leaf': 12, 'min_samples_split': 2, 'n_estimators': 50}",0.725341
5,"{'max_depth': 7, 'min_samples_leaf': 8, 'min_samples_split': 2, 'n_estimators': 50}",0.725413
6,"{'max_depth': 7, 'min_samples_leaf': 9, 'min_samples_split': 2, 'n_estimators': 50}",0.725484
7,"{'max_depth': 7, 'min_samples_leaf': 10, 'min_samples_split': 2, 'n_estimators': 50}",0.725533
8,"{'max_depth': 7, 'min_samples_leaf': 11, 'min_samples_split': 2, 'n_estimators': 50}",0.725489
9,"{'max_depth': 7, 'min_samples_leaf': 12, 'min_samples_split': 2, 'n_estimators': 50}",0.725386


In [18]:
my_grid_search.best_score_

0.7255327801710438

In [19]:
RF_optimized_parms = my_grid_search.best_params_
RF_optimized_parms

{'max_depth': 7,
 'min_samples_leaf': 10,
 'min_samples_split': 2,
 'n_estimators': 50}

In [20]:
train_scores = [x[1] for x in my_grid_search.predict_proba(total_df_piped)]
roc_auc_score(total_df['TARGET'], train_scores)

0.7360699554273631

In [21]:
del my_grid_search

### DONE. random forest grid search, all features

In [22]:
total_df_piped, final_feature_list, total_pipeline, trans_list = master_pipeline(df_in = total_df,
                                                                             int_cutoff=20, 
                                                                             poly_deg=4, 
                                                                             feats_with_interaction=[]
                                                                               )


Total original feat len is 85. Sum of feats is 85.
Final array is length 417. final feature list is length 417.


In [23]:
param_dist_dict =  { 'max_depth' :         [14,15,16,17,18],
                     'min_samples_leaf' :  [61,62,63,64,65],
                     'min_samples_split' : [2], 
                     'n_estimators':       [40],
                    }

forest_reg = RandomForestClassifier(random_state=0,
                                    class_weight = None)

my_grid_search = GridSearchCV(forest_reg, 
                                param_dist_dict, 
                                cv = 3,
                                return_train_score = True , 
                                scoring = 'roc_auc',
                                n_jobs = -1, 
                                refit = True,
                                verbose = 51, 
                                        )

my_grid_search.fit(total_df_piped, total_df['TARGET'])

Fitting 3 folds for each of 25 candidates, totalling 75 fits
Memmaping (shape=(73187618,), dtype=int32) to new file /dev/shm/joblib_memmaping_pool_3496_140630285815200/3496-140630235841704-9bf90f5320b696a352c12cd4001de90f.pkl
Memmaping (shape=(307512,), dtype=int32) to new file /dev/shm/joblib_memmaping_pool_3496_140630285815200/3496-140630235841704-851f25297a32c2e0d1a0ef9235730f5f.pkl
Memmaping (shape=(73187618,), dtype=float64) to new file /dev/shm/joblib_memmaping_pool_3496_140630285815200/3496-140630235841704-94dce670310afaed17a34e987de612cb.pkl
[CV] max_depth=14, min_samples_leaf=61, min_samples_split=2, n_estimators=40 
Memmaping (shape=(307511,), dtype=int64) to new file /dev/shm/joblib_memmaping_pool_3496_140630285815200/3496-140630235841704-36f41ebc26c9047822d0779f2962f9e3.pkl
Memmaping (shape=(307511,), dtype=int64) to new file /dev/shm/joblib_memmaping_pool_3496_140630285815200/3496-140630235841704-7fee98903a11851f0c36b343c7ef5338.pkl
Memmaping (shape=(205007,), dtype=int64)

[CV]  max_depth=15, min_samples_leaf=64, min_samples_split=2, n_estimators=40, score=0.7500110468810652, total= 5.4min
[CV] max_depth=16, min_samples_leaf=61, min_samples_split=2, n_estimators=40 
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed: 21.5min
[CV]  max_depth=15, min_samples_leaf=64, min_samples_split=2, n_estimators=40, score=0.752061224000358, total= 5.4min
[CV]  max_depth=15, min_samples_leaf=64, min_samples_split=2, n_estimators=40, score=0.7468440729611627, total= 5.4min
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed: 21.5min
[Parallel(n_jobs=-1)]: Done  27 tasks      | elapsed: 21.5min
[CV] max_depth=16, min_samples_leaf=62, min_samples_split=2, n_estimators=40 
[CV] max_depth=16, min_samples_leaf=62, min_samples_split=2, n_estimators=40 
[CV]  max_depth=15, min_samples_leaf=65, min_samples_split=2, n_estimators=40, score=0.74590406049106, total= 5.3min
[CV] max_depth=16, min_samples_leaf=62, min_samples_split=2, n_estimators=40 
[Parallel(n_jobs=-1)]: Done  

[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed: 40.0min
[CV]  max_depth=17, min_samples_leaf=65, min_samples_split=2, n_estimators=40, score=0.750026306998644, total= 5.7min
[CV] max_depth=18, min_samples_leaf=62, min_samples_split=2, n_estimators=40 
[Parallel(n_jobs=-1)]: Done  57 tasks      | elapsed: 44.7min
[CV]  max_depth=17, min_samples_leaf=64, min_samples_split=2, n_estimators=40, score=0.751035016050931, total= 5.8min
[Parallel(n_jobs=-1)]: Done  58 tasks      | elapsed: 44.8min
[CV] max_depth=18, min_samples_leaf=62, min_samples_split=2, n_estimators=40 
[CV]  max_depth=17, min_samples_leaf=65, min_samples_split=2, n_estimators=40, score=0.7464385057434965, total= 5.7min
[CV] max_depth=18, min_samples_leaf=63, min_samples_split=2, n_estimators=40 
[Parallel(n_jobs=-1)]: Done  59 tasks      | elapsed: 44.8min
[CV]  max_depth=17, min_samples_leaf=65, min_samples_split=2, n_estimators=40, score=0.7521093902595581, total= 5.7min
[CV] max_depth=18, min_samples_leaf=63, min_

GridSearchCV(cv=3, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_depth': [14, 15, 16, 17, 18], 'min_samples_leaf': [61, 62, 63, 64, 65], 'min_samples_split': [2], 'n_estimators': [40]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=51)

In [24]:
pd.set_option('display.max_colwidth', -1)
pd.DataFrame(my_grid_search.cv_results_)[['params','mean_test_score']]

,params,mean_test_score
0,"{'max_depth': 14, 'min_samples_leaf': 61, 'min_samples_split': 2, 'n_estimators': 40}",0.749026
1,"{'max_depth': 14, 'min_samples_leaf': 62, 'min_samples_split': 2, 'n_estimators': 40}",0.749254
2,"{'max_depth': 14, 'min_samples_leaf': 63, 'min_samples_split': 2, 'n_estimators': 40}",0.749186
3,"{'max_depth': 14, 'min_samples_leaf': 64, 'min_samples_split': 2, 'n_estimators': 40}",0.749308
4,"{'max_depth': 14, 'min_samples_leaf': 65, 'min_samples_split': 2, 'n_estimators': 40}",0.748971
5,"{'max_depth': 15, 'min_samples_leaf': 61, 'min_samples_split': 2, 'n_estimators': 40}",0.749518
6,"{'max_depth': 15, 'min_samples_leaf': 62, 'min_samples_split': 2, 'n_estimators': 40}",0.749568
7,"{'max_depth': 15, 'min_samples_leaf': 63, 'min_samples_split': 2, 'n_estimators': 40}",0.749655
8,"{'max_depth': 15, 'min_samples_leaf': 64, 'min_samples_split': 2, 'n_estimators': 40}",0.749639
9,"{'max_depth': 15, 'min_samples_leaf': 65, 'min_samples_split': 2, 'n_estimators': 40}",0.749503


In [25]:
RF_optimized_parms = my_grid_search.best_params_
RF_optimized_parms

{'max_depth': 15,
 'min_samples_leaf': 63,
 'min_samples_split': 2,
 'n_estimators': 40}

In [26]:
my_grid_search.best_score_

0.7496549981982892

In [27]:
#pd.DataFrame(my_grid_search.cv_results_)

In [28]:
train_scores = [x[1] for x in my_grid_search.predict_proba(total_df_piped)]
roc_auc_score(total_df['TARGET'], train_scores)

0.8498758422290261

In [29]:
del my_grid_search

### DONE. log reg grid search, on EXT sources with poly interactions

In [30]:
total_df_piped, final_feature_list, total_pipeline, trans_list = \
master_pipeline(df_in = total_df[['EXT_SOURCE_1','EXT_SOURCE_2',
                                         'EXT_SOURCE_3']],
                        int_cutoff=20, 
                        poly_deg=4, 
                        feats_with_interaction=['EXT_SOURCE_1','EXT_SOURCE_2',
                                               'EXT_SOURCE_3']
               )


Total original feat len is 3. Sum of feats is 3.
Final array is length 35. final feature list is length 35.


In [32]:
param_grid = [{ 'C' : [27.5,28,28.5,29,29.5,30,30.5,31] }]

my_LgRg = LogisticRegression(penalty= 'l2', 
                            random_state = 0, 
                            class_weight = None) 

my_LgRg_grid_search = GridSearchCV(my_LgRg, 
                                   param_grid, 
                                   cv = 3, # for integer inputs, Stratified folding is default. 
                                   return_train_score = True , 
                                   scoring = 'roc_auc',
                                   n_jobs = -1, 
                                   refit = True,
                                   verbose = 51, 
                                    )

my_LgRg_grid_search.fit(total_df_piped, 
                        total_df['TARGET'])

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] C=27.5 ..........................................................
Memmaping (shape=(307511, 35), dtype=float64) to new file /dev/shm/joblib_memmaping_pool_3496_140630282662296/3496-140630235858760-b675758a20e22f3b4e24b10b420720d1.pkl
Memmaping (shape=(307511,), dtype=int64) to new file /dev/shm/joblib_memmaping_pool_3496_140630282662296/3496-140630235858760-36f41ebc26c9047822d0779f2962f9e3.pkl
Memmaping (shape=(307511,), dtype=int64) to new file /dev/shm/joblib_memmaping_pool_3496_140630282662296/3496-140630235858760-7fee98903a11851f0c36b343c7ef5338.pkl
Memmaping (shape=(205007,), dtype=int64) to new file /dev/shm/joblib_memmaping_pool_3496_140630282662296/3496-140630235858760-981b55e712abb5a5a2369212f2856f0a.pkl
[CV] C=27.5 ..........................................................
[CV] C=27.5 ..........................................................
Memmaping (shape=(205007,), dtype=int64) to new file /dev/shm/joblib_m

GridSearchCV(cv=3, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid=[{'C': [27.5, 28, 28.5, 29, 29.5, 30, 30.5, 31]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=51)

In [33]:
pd.set_option('display.max_colwidth', -1)
pd.DataFrame(my_LgRg_grid_search.cv_results_)[['params','mean_test_score']]

,params,mean_test_score
0,{'C': 27.5},0.725642
1,{'C': 28},0.725642
2,{'C': 28.5},0.725642
3,{'C': 29},0.725642
4,{'C': 29.5},0.725641
5,{'C': 30},0.725641
6,{'C': 30.5},0.725642
7,{'C': 31},0.725642


In [34]:
LgRg_optimized_parms = my_LgRg_grid_search.best_params_
my_LgRg_grid_search.best_params_

{'C': 30.5}

In [35]:
my_LgRg_grid_search.best_score_

0.7256419872244535

In [36]:
#pd.DataFrame(my_LgRg_grid_search.cv_results_)

In [37]:
train_scores = [x[1] for x in my_LgRg_grid_search.predict_proba(total_df_piped)]
roc_auc_score(total_df['TARGET'], train_scores)

0.7258789804884163

In [38]:
del my_LgRg_grid_search

### DONE. log reg grid search, on all features

In [39]:
total_df_piped, final_feature_list, total_pipeline, trans_list = master_pipeline(df_in = total_df,
                                                                                         int_cutoff=20, 
                                                                                         poly_deg=4, 
                                                                                         feats_with_interaction=[])

Total original feat len is 85. Sum of feats is 85.
Final array is length 417. final feature list is length 417.


In [40]:
param_grid = [{ 'C' : [4,5,5.5,6,6.5,7,8,9] }]

my_LgRg = LogisticRegression(penalty= 'l2', 
                            random_state = 0, 
                            class_weight = None) 

my_LgRg_grid_search = GridSearchCV(my_LgRg, 
                           param_grid, 
                           cv = 3, # for integer inputs, Stratified folding is default. 
                           return_train_score = True , 
                           scoring = 'roc_auc',
                           n_jobs = -1, 
                           refit = True,
                           verbose = 51,
                            )

my_LgRg_grid_search.fit(total_df_piped, 
                        total_df['TARGET'])

Fitting 3 folds for each of 8 candidates, totalling 24 fits
Memmaping (shape=(73187618,), dtype=int32) to new file /dev/shm/joblib_memmaping_pool_3496_140631203124336/3496-140630279311544-9bf90f5320b696a352c12cd4001de90f.pkl
Memmaping (shape=(307512,), dtype=int32) to new file /dev/shm/joblib_memmaping_pool_3496_140631203124336/3496-140630279311544-851f25297a32c2e0d1a0ef9235730f5f.pkl
Memmaping (shape=(73187618,), dtype=float64) to new file /dev/shm/joblib_memmaping_pool_3496_140631203124336/3496-140630279311544-94dce670310afaed17a34e987de612cb.pkl
[CV] C=4 .............................................................
Memmaping (shape=(307511,), dtype=int64) to new file /dev/shm/joblib_memmaping_pool_3496_140631203124336/3496-140630279311544-36f41ebc26c9047822d0779f2962f9e3.pkl
Memmaping (shape=(307511,), dtype=int64) to new file /dev/shm/joblib_memmaping_pool_3496_140631203124336/3496-140630279311544-7fee98903a11851f0c36b343c7ef5338.pkl
Memmaping (shape=(205007,), dtype=int64) to new 

GridSearchCV(cv=3, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid=[{'C': [4, 5, 5.5, 6, 6.5, 7, 8, 9]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=51)

In [41]:
pd.set_option('display.max_colwidth', -1)
pd.DataFrame(my_LgRg_grid_search.cv_results_)[['params','mean_test_score']]

,params,mean_test_score
0,{'C': 4},0.767527
1,{'C': 5},0.767524
2,{'C': 5.5},0.767550
3,{'C': 6},0.767523
4,{'C': 6.5},0.767549
5,{'C': 7},0.767544
6,{'C': 8},0.767481
7,{'C': 9},0.767499


In [42]:
my_LgRg_grid_search.best_score_

0.7675500818243326

In [43]:
LgRg_optimized_parms = my_LgRg_grid_search.best_params_
LgRg_optimized_parms

{'C': 5.5}

In [44]:
train_scores = [x[1] for x in my_LgRg_grid_search.predict_proba(total_df_piped)]
roc_auc_score(total_df['TARGET'], train_scores)

0.7709731136997777

In [45]:
del my_LgRg_grid_search

### DONE. MLP Classifier, all features. 

In [23]:
total_df_piped, final_feature_list, total_pipeline, trans_list = master_pipeline(df_in = total_df,
                                                                                         int_cutoff=20, 
                                                                                         poly_deg=4, 
                                                                                         feats_with_interaction=[])

Total original feat len is 85. Sum of feats is 85.
Final array is length 417. final feature list is length 417.


In [24]:
param_dist_dict =  { 'alpha' :         [.34,.35,.36],
                    'hidden_layer_sizes' : [(96, ),(97, ),(98, ),(99, ),(100, ),(101, ),(102, )],
                    }

In [25]:
my_MLP = MLPClassifier(random_state=0, 
                       tol=0.0001, 
                     #  verbose=51, 
                       warm_start=False, 
                       momentum=0.9)


my_grid_search = GridSearchCV(my_MLP, 
                                param_dist_dict, 
                                cv = 3,
                                return_train_score = True , 
                                scoring = 'roc_auc',
                                n_jobs = -1, 
                                refit = True,
                                verbose = 51, 
                                        )

my_grid_search.fit(total_df_piped, total_df['TARGET'])

Fitting 3 folds for each of 21 candidates, totalling 63 fits
Memmaping (shape=(73187618,), dtype=int32) to new file /dev/shm/joblib_memmaping_pool_1835_139712370625784/1835-139712359562656-9bf90f5320b696a352c12cd4001de90f.pkl
Memmaping (shape=(307512,), dtype=int32) to new file /dev/shm/joblib_memmaping_pool_1835_139712370625784/1835-139712359562656-851f25297a32c2e0d1a0ef9235730f5f.pkl
Memmaping (shape=(73187618,), dtype=float64) to new file /dev/shm/joblib_memmaping_pool_1835_139712370625784/1835-139712359562656-94dce670310afaed17a34e987de612cb.pkl
[CV] alpha=0.34, hidden_layer_sizes=(96,) ............................
Memmaping (shape=(307511,), dtype=int64) to new file /dev/shm/joblib_memmaping_pool_1835_139712370625784/1835-139712359562656-36f41ebc26c9047822d0779f2962f9e3.pkl
Memmaping (shape=(307511,), dtype=int64) to new file /dev/shm/joblib_memmaping_pool_1835_139712370625784/1835-139712359562656-7fee98903a11851f0c36b343c7ef5338.pkl
Memmaping (shape=(205007,), dtype=int64) to new

[CV]  alpha=0.35, hidden_layer_sizes=(99,), score=0.7638376711010267, total=11.6min
[CV] alpha=0.35, hidden_layer_sizes=(101,) ...........................
[Parallel(n_jobs=-1)]: Done  30 tasks      | elapsed: 77.1min
[CV]  alpha=0.35, hidden_layer_sizes=(99,), score=0.7587675085690676, total=13.1min
[CV] alpha=0.35, hidden_layer_sizes=(101,) ...........................
[Parallel(n_jobs=-1)]: Done  31 tasks      | elapsed: 77.2min
[CV]  alpha=0.35, hidden_layer_sizes=(98,), score=0.7615002764205054, total=21.9min
[CV] alpha=0.35, hidden_layer_sizes=(102,) ...........................
[Parallel(n_jobs=-1)]: Done  32 tasks      | elapsed: 79.6min
[CV]  alpha=0.35, hidden_layer_sizes=(101,), score=0.7594786975061942, total=13.0min
[CV] alpha=0.35, hidden_layer_sizes=(102,) ...........................
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 91.1min
[CV]  alpha=0.35, hidden_layer_sizes=(99,), score=0.7592330909218107, total=25.2min
[Parallel(n_jobs=-1)]: Done  34 tasks      | el

GridSearchCV(cv=3, error_score='raise',
       estimator=MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=0, shuffle=True,
       solver='adam', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'alpha': [0.34, 0.35, 0.36], 'hidden_layer_sizes': [(96,), (97,), (98,), (99,), (100,), (101,), (102,)]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=51)

In [26]:
pd.set_option('display.max_colwidth', -1)
pd.DataFrame(my_grid_search.cv_results_)[['params','mean_test_score']]

,params,mean_test_score
0,"{'alpha': 0.34, 'hidden_layer_sizes': (96,)}",0.761496
1,"{'alpha': 0.34, 'hidden_layer_sizes': (97,)}",0.760227
2,"{'alpha': 0.34, 'hidden_layer_sizes': (98,)}",0.760975
3,"{'alpha': 0.34, 'hidden_layer_sizes': (99,)}",0.761682
4,"{'alpha': 0.34, 'hidden_layer_sizes': (100,)}",0.762536
5,"{'alpha': 0.34, 'hidden_layer_sizes': (101,)}",0.761940
6,"{'alpha': 0.34, 'hidden_layer_sizes': (102,)}",0.761445
7,"{'alpha': 0.35, 'hidden_layer_sizes': (96,)}",0.760107
8,"{'alpha': 0.35, 'hidden_layer_sizes': (97,)}",0.761491
9,"{'alpha': 0.35, 'hidden_layer_sizes': (98,)}",0.759639


In [27]:
my_grid_search.best_score_

0.7625392362887993

In [28]:
my_grid_search_optimized_parms = my_grid_search.best_params_
my_grid_search_optimized_parms

{'alpha': 0.35, 'hidden_layer_sizes': (100,)}

In [29]:
train_scores = [x[1] for x in my_grid_search.predict_proba(total_df_piped)]
roc_auc_score(total_df['TARGET'], train_scores)

0.7630282799516954

In [30]:
del my_grid_search